In [1]:
from datetime import datetime
from packaging import version

import os
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)


TensorFlow version:  2.3.0


In [2]:
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [3]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()


In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(128)


In [6]:
ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(128)


In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)


In [8]:
# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

model.fit(ds_train,
          epochs=2,
          validation_data=ds_test,
          callbacks = [tboard_callback])


Epoch 1/2
469/469 [==============================] - 6s 12ms/step - loss: 0.3648 - accuracy: 0.8977 - val_loss: 0.2009 - val_accuracy: 0.9413
Epoch 2/2
 48/469 [==>...........................] - ETA: 3s - loss: 0.2050 - accuracy: 0.9414WARNING:tensorflow:From /home/phuong/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


469/469 [==============================] - 5s 11ms/step - loss: 0.1720 - accuracy: 0.9517 - val_loss: 0.1417 - val_accuracy: 0.9604


In [9]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [11]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs --host localhost

Reusing TensorBoard on port 6006 (pid 11621), started 0:00:17 ago. (Use '!kill 11621' to kill it.)